In [ ]:
from scripts2 import *
from scripts import *

import warnings
warnings.filterwarnings('ignore')

import copy

In [ ]:
path = 'analysis-results/experimental-results/'

In [ ]:
path_sign = 'analysis-results/significance/'

# Choosing a dataset by index

In [ ]:
data_order

In [ ]:
i = 0

In [ ]:
dataset_name = data_order[i]
print(dataset_name)

In [ ]:
data = datasets_load[dataset_name]
n_classes = len(data['target_names'])
print('Number of classes {}'.format(n_classes))

Loading results for all experiments

In [ ]:
results = {}
for algo in algo_arr:
    all_dfs = []
    for seed in range(1, 11):
        file_name = path + dataset_name + '_' + algo + '_' + str(seed) + '.csv'
        df = pd.read_csv(file_name)
        df['fold'] = seed
        all_dfs.append(df)
        pass
    
    df = pd.concat(all_dfs)
    df.name = algo
    results[algo] = df
    pass

## Distribution between classes

In [ ]:
x, y = np.unique(data['target'], return_counts=True)
print(y)
plt.bar(x, y)
plt.title('Distribution between classes for {} dataset'.format(dataset_name))

## Visualization for synthetic datasets

In [ ]:
if dataset_name.startswith('gen'):
    col_arr = ['r', 'g', 'y', 'b']
    for i in range(0,4):
        # plot every 5th point to see something
        plt.scatter(data['data'].T[0][i*2000: (i+1)*2000][::5], 
                    data['data'].T[1][i*2000: (i+1)*2000][::5], s=1,
                    c=col_arr[i]
                    )
    plt.title('$\sigma=1.0$')
    plt.grid(True)
    #tikzplotlib.save('../paper/sample-papers/plots/{}-class-visualization.tex'.format(
    #    dataset_name.replace('.', '').replace('_', '-')))

# Eps analysis

Sometimes there is small deviation from $\epsilon$. All cases even with smallest differences are shown below.

In [ ]:
all_algos = []

func_arr = ['marg_err', 'inv_err', 'inv_m_err']

for algo in algo_arr:
    print(algo)
    df = results[algo]
    #print(df.name)
    df = df[['eps', 'marg_err', 'inv_err', 'inv_m_err']].groupby(['eps']).mean().copy()
    #df.to_csv(path + 'A_significance_{}.csv'.format(dataset_name))
    df['algo'] = algo
    df.reset_index(inplace=True)
    for func_name in func_arr:
        if sum(df[func_name] <= df['eps']) < 6:
            print('\tNot calibrated {} for eps = {}'.format(func_name,
                                                           df[df[func_name] > df['eps']]['eps'].values))
    all_algos.append(df)
    pass
df = pd.concat(all_algos)
df.reset_index(inplace=True, drop=True)
df.to_csv(path_sign + 'A_significance_{}.csv'.format(dataset_name))

# Baseline error

In [ ]:
plot_baseline_err(algo_arr, results, eps_err)

# How many cases deviate by threshold?

In [ ]:
all_res_df = []
for algo in algo_arr:
    df = results[algo].groupby('eps').mean()
    df.reset_index(inplace=True)
    df.drop(df[df['eps'] == 0.03].index, inplace=True)
    df['algo'] = algo
    all_res_df.append(df) 
all_res_df = pd.concat(all_res_df)
all_res_df.reset_index(inplace=True, drop=True)

cols = ['marg_oneC', 'inv_oneC', 'inv_m_oneC']
all_res_df['diff_oneC'] = all_res_df[cols].max(axis=1) - all_res_df[cols].min(axis=1)

cols = ['marg_avgC', 'inv_avgC', 'inv_m_avgC']
all_res_df['diff_avgC'] = (all_res_df[cols].max(axis=1) - all_res_df[cols].min(axis=1)) / n_classes

all_res_df.head()

In [ ]:
all_res_df[['diff_oneC', 'diff_avgC']].describe()

Set a threshold here (fraction of 1)

In [ ]:
threshold = 0.02

In [ ]:
oneC_above = (all_res_df['diff_oneC'] > threshold).sum() / len(all_res_df) * 100
print('for oneC {}% of cases differ at least by {}%'.format(oneC_above, threshold * 100))
#oneC_above

In [ ]:
avgC_above = (all_res_df['diff_avgC'] > threshold).sum() / len(all_res_df) * 100
print('for avgC {}% of cases differ at least by {}%'.format(avgC_above, threshold * 100))
#avgC_above

# oneC vs E_oneC

In [ ]:
all_res_df['eff_oneC_coef_m'] = all_res_df['marg_eff_oneC'] / all_res_df['marg_oneC']
all_res_df['eff_oneC_coef_ip_m'] = all_res_df['inv_m_eff_oneC'] / all_res_df['inv_m_oneC']
all_res_df['eff_oneC_coef_ip'] = all_res_df['inv_eff_oneC'] / all_res_df['inv_oneC']
#all_res_df.head()

In [ ]:
eff_oneC_df = all_res_df[['algo', 'eff_oneC_coef_m', 'eff_oneC_coef_ip_m', 'eff_oneC_coef_ip']].groupby('algo').mean()
#eff_oneC_df

In [ ]:
eff_oneC_df.mean(axis=1)

In [ ]:
eff_oneC_df.std(axis=1)

In [ ]:
## functions
def get_baseline_accuracy(algo_arr, results, eps_err):
    baseline_err_arr = []
    for algo in algo_arr:
        df = results[algo]
        baseline_err_arr.append(1 - df[df['eps'] == eps_err[0]]['origin_err'].mean())

    return pd.DataFrame({'algo': algo_arr, 'b_err': baseline_err_arr})

In [ ]:
baseline_acc = get_baseline_accuracy(algo_arr, results, eps_err)

In [ ]:
corr_df = pd.DataFrame({
    'acc' : baseline_acc.sort_values('algo')['b_err'].values,
    'oneC' : eff_oneC_df.mean(axis=1).values
})

eff_oneC_corr = np.round(corr_df.corr().values[0,1], 3)
print('Correlation between baseline accuracy and oneC = {}'.format(eff_oneC_corr))

In [ ]:
corr_df

In [ ]:
corr_df.plot(
    kind='line'
)

# Analysis of oneC

In [ ]:
stat_res_dic = {}
for algo in algo_arr:
    sub_dic = {}
    stat_res_dic[algo] = {'oneC':{}, 'avgC':{}, 'eff_oneC':{}}
#stat_res_dic

In [ ]:
col = 'oneC'
col_str = '_{}'.format(col)
plot_metric(results, col_str, nc_func_arr, algo_arr, eps_err, colours, ls_arr, lw_arr)

Plotting into files for paper

In [ ]:
"""
# find max and min for plotting
min_val = np.inf
max_val = 0
for algo in algo_arr:
    new_min = np.min(np.min(results[algo][['eps', 'marg{}'.format(col_str),
                                           'inv{}'.format(col_str),
                                           'inv_m{}'.format(col_str)]].groupby('eps').mean()))
    new_max = np.max(np.max(results[algo][['eps', 'marg{}'.format(col_str),
                                           'inv{}'.format(col_str),
                                           'inv_m{}'.format(col_str)]].groupby('eps').mean()))
    if new_min < min_val:
        min_val = new_min
    if new_max > max_val:
        max_val = new_max
min_val = min(min_val, -0.05)
print('{} - {}'.format(min_val, max_val))


lw_arr_2 = [6, 6, 1]

for k in [0, 2, 4, 6]:
    plt.clf()
    for i in [k, k+1]:
        algo = algo_arr[i]
        metric_df = results[algo][['eps', 'marg{}'.format(col_str),
                                   'inv{}'.format(col_str),
                                   'inv_m{}'.format(col_str)]].groupby('eps').mean()

        for nc_idx in range(0, len(nc_func_arr)):
            # print(nc_idx)
            if nc_idx == 2:
                label_str = algo
            else:
                label_str = '_nolegend_'
            plt.plot(eps_err, metric_df[nc_func_arr[nc_idx] + col_str],
                     c=colours[i], ls=ls_arr[nc_idx], lw=lw_arr[nc_idx],
                     label=label_str
                     )
        pass
    plt.xticks(eps_err)
    plt.ylim((min_val * 0.9, max_val * 1.1))
    plt.grid(True)
    plt.legend()
    tikzplotlib.save('../paper/sample-papers/plots/{}-oneC-{}.tex'.format(dataset_name,
                                                                         k))
    pass
"""

## oneC significance of differences

### SVM

In [ ]:
algo = algo_arr[0]

In [ ]:
mean_res = mean_test_for_algo(algo, results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo, results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo, results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### DT

In [ ]:
algo = algo_arr[1]

In [ ]:
mean_res = mean_test_for_algo(algo, results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo, results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo, results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### KNN

In [ ]:
algo = algo_arr[2]

In [ ]:
mean_res = mean_test_for_algo(algo, results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[2], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[2], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### Ada

In [ ]:
algo = algo_arr[3]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[3], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[3], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[3], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### GNB

In [ ]:
algo = algo_arr[4]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[4], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[4], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[4], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### MPR

In [ ]:
algo = algo_arr[5]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[5], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[5], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[5], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### RF

In [ ]:
algo = algo_arr[6]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[6], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[6], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[6], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### QDA

In [ ]:
algo = algo_arr[7]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[7], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[7], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[7], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

# eff_oneC
$eff\_oneC = \frac{number\ \ of\ \ correst\ \ singleton\ \ predictions}{total\ \ number\ \ of\ \ intsnaces}$

In [ ]:
col = 'eff_oneC'
col_str = '_{}'.format(col)
plot_metric(results, col_str, nc_func_arr, algo_arr, eps_err, colours, ls_arr, lw_arr)

## eff_oneC stat

### SVM

In [ ]:
stat_res = stats_test_for_algo(algo_arr[0], results, col, p_val=p_val)

### DT

In [ ]:
stat_res = stats_test_for_algo(algo_arr[1], results, col, p_val=p_val)

### KNN

In [ ]:
stat_res = stats_test_for_algo(algo_arr[2], results, col, p_val=p_val)

### Ada

In [ ]:
stat_res = stats_test_for_algo(algo_arr[3], results, col, p_val=p_val)

### GNB

In [ ]:
stat_res = stats_test_for_algo(algo_arr[4], results, col, p_val=p_val)

### MPR

In [ ]:
stat_res = stats_test_for_algo(algo_arr[5], results, col, p_val=p_val)

### RF

In [ ]:
stat_res = stats_test_for_algo(algo_arr[6], results, col, p_val=p_val)

### QDA

In [ ]:
stat_res = stats_test_for_algo(algo_arr[7], results, col, p_val=p_val)

# avg_C

In [ ]:
col = 'avgC'
col_str = '_{}'.format(col)
plot_metric(results, col_str, nc_func_arr, algo_arr, eps_err, colours, ls_arr, lw_arr)

Plotting into files for paper

In [ ]:
"""
# find max and min for plottin
min_val = np.inf
max_val = 0
for algo in algo_arr:
    new_min = np.min(np.min(results[algo][['eps', 'marg{}'.format(col_str),
                                           'inv{}'.format(col_str),
                                           'inv_m{}'.format(col_str)]].groupby('eps').mean()))
    new_max = np.max(np.max(results[algo][['eps', 'marg{}'.format(col_str),
                                           'inv{}'.format(col_str),
                                           'inv_m{}'.format(col_str)]].groupby('eps').mean()))
    if new_min < min_val:
        min_val = new_min
    if new_max > max_val:
        max_val = new_max
min_val = min(min_val, -0.05)
print('{} - {}'.format(min_val, max_val))

for k in [0, 2, 4, 6]:
    plt.clf()
    for i in [k, k+1]:
        algo = algo_arr[i]
        metric_df = results[algo][['eps', 'marg{}'.format(col_str),
                                   'inv{}'.format(col_str),
                                   'inv_m{}'.format(col_str)]].groupby('eps').mean()

        for nc_idx in range(0, len(nc_func_arr)):
            # print(nc_idx)
            if nc_idx == 2:
                label_str = algo
            else:
                label_str = '_nolegend_'
            plt.plot(eps_err, metric_df[nc_func_arr[nc_idx] + col_str],
                     c=colours[i], ls=ls_arr[nc_idx], lw=lw_arr[nc_idx],
                     label=label_str
                     )
        pass
    plt.xticks(eps_err)
    plt.ylim((min_val * 0.9, max_val * 1.1))
    plt.grid(True)
    plt.legend()
    tikzplotlib.save('../paper/sample-papers/plots/{}-{}-{}.tex'.format(dataset_name,
                                                                        col,
                                                                         k))
    pass
"""

## avgC stat

### SVM

In [ ]:
algo = algo_arr[0]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[0], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[0], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[0], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### DT

In [ ]:
algo = algo_arr[1]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[1], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[1], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[1], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### KNN

In [ ]:
algo = algo_arr[2]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[2], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[2], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[2], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### Ada

In [ ]:
algo = algo_arr[3]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[3], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[3], results, col,  n_classes, threshold)

In [ ]:
stat_res = stat_res = stats_test_for_algo(algo_arr[3], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### GNB

In [ ]:
algo = algo_arr[4]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[4], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[4], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[4], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### MPR

In [ ]:
algo = algo_arr[5]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[5], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[5], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[5], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### RF

In [ ]:
algo = algo_arr[6]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[6], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[6], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[6], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

### QDA

In [ ]:
algo = algo_arr[7]

In [ ]:
mean_res = mean_test_for_algo(algo_arr[7], results, col)

In [ ]:
mean_res_threshold = mean_threshold_test_for_algo(algo_arr[7], results, col,  n_classes, threshold)

In [ ]:
stat_res = stats_test_for_algo(algo_arr[7], results, col, p_val=p_val)

In [ ]:
stat_res_dic[algo][col]= (copy.deepcopy(mean_res), 
                          copy.deepcopy(mean_res_threshold), 
                          copy.deepcopy(stat_res))

# Analysis of patterns

In [ ]:
mean_res_val = 0
mean_res_th_val = 1
stat_res_val = 2

func_short_arr = ['IP', 'IP_M', 'M']
ip = 'IP'
ip_m = 'IP_M'
m = 'M'

In [ ]:
main_pattern = {}
main_pattern_oneC = {}
main_pattern_avgC = {}
inverse_pattern = {}
margin_best = {}
ip_best = {}
ip_m_best = {}
ip_not_expected = {}
margin_better_ip = {}
patterns_dic = {
    'is_main_pattern': main_pattern,
    'is_main_pattern_oneC': main_pattern_oneC,
    'is_main_pattern_avgC': main_pattern_avgC,
    'is_inverse_pattern': inverse_pattern,
    'is_margin_best': margin_best,
    'is_ip_best': ip_best,
    'is_ip_m_best': ip_m_best,
    'is_ip_not_expected': ip_not_expected,
    'is_margin_better_ip': margin_better_ip,
}


def add_val(patterns_dic, key, eps, algo):
    if algo in patterns_dic[key]:
        patterns_dic[key][algo].append(eps)
    else:
        patterns_dic[key][algo] = [eps]
    pass


for algo in algo_arr:
    for eps in eps_err:
        if eps == 0.03:
            continue
        # condence oneC
        oneC_res_th_df = stat_res_dic[algo]['oneC'][mean_res_th_val][str(eps)]
        oneC_stat_df = stat_res_dic[algo]['oneC'][stat_res_val][str(eps)]
        oneC_df = pd.DataFrame({})
        oneC_df['n-func'] = func_short_arr
        for func_short in func_short_arr:
            th_arr = oneC_res_th_df[func_short].values
            st_arr = oneC_stat_df[func_short].values
            final_arr = []
            for i in range(0, 3):
                if np.isnan(th_arr[i]):
                    final_arr.append(st_arr[i])
                else:
                    final_arr.append(th_arr[i])
            oneC_df[func_short] = final_arr
        oneC_df.set_index('n-func', inplace=True)

        # condence avgC
        avgC_res_th_df = stat_res_dic[algo]['avgC'][mean_res_th_val][str(eps)]
        avgC_stat_df = stat_res_dic[algo]['avgC'][stat_res_val][str(eps)]

        avgC_df = pd.DataFrame({})
        avgC_df['n-func'] = func_short_arr
        for func_short in func_short_arr:
            th_arr = avgC_res_th_df[func_short].values
            st_arr = avgC_stat_df[func_short].values
            final_arr = []
            for i in range(0, 3):
                if np.isnan(th_arr[i]):
                    final_arr.append(st_arr[i])
                else:
                    final_arr.append(th_arr[i])
            avgC_df[func_short] = final_arr
        avgC_df.set_index('n-func', inplace=True)

        is_main_pattern = ((oneC_df.T[m][ip] == 1) and (avgC_df.T[ip][m] == 1))
        if is_main_pattern:
            key = 'is_main_pattern'
            add_val(patterns_dic, key, eps, algo)

        is_main_pattern_oneC = (oneC_df.T[m][ip] == 1) and np.isnan(avgC_df.T[ip][m])
        if is_main_pattern_oneC:
            key = 'is_main_pattern_oneC'
            add_val(patterns_dic, key, eps, algo)

        is_main_pattern_avgC = np.isnan(oneC_df.T[m][ip]) and (avgC_df.T[ip][m] == 1)
        if is_main_pattern_avgC:
            key = 'is_main_pattern_avgC'
            add_val(patterns_dic, key, eps, algo)

        is_inverse_pattern = (oneC_df.T[ip][m] == 1) and (avgC_df.T[m][ip] == 1)
        if is_inverse_pattern:
            key = 'is_inverse_pattern'
            add_val(patterns_dic, key, eps, algo)

        is_margin_best = (~np.isnan(oneC_df.T[m][ip]) or ~np.isnan(oneC_df.T[m][ip_m]) or ~np.isnan(
            avgC_df.T[m][ip]) or ~np.isnan(avgC_df.T[m][ip_m])) and (
                                 ((oneC_df.T[m][ip] == 1) or np.isnan(oneC_df.T[m][ip])) and (
                                 (oneC_df.T[m][ip_m] == 1) or np.isnan(oneC_df.T[m][ip_m])) and (
                                         (avgC_df.T[m][ip] == 1) or (avgC_df.T[m][ip])) and (
                                         (avgC_df.T[m][ip_m] == 1) or np.isnan(avgC_df.T[m][ip_m])))
        if is_margin_best:
            key = 'is_margin_best'
            add_val(patterns_dic, key, eps, algo)

        is_ip_best = (~np.isnan(oneC_df.T[ip][m]) or ~np.isnan(oneC_df.T[ip][ip_m]) or ~np.isnan(
            avgC_df.T[ip][m]) or ~np.isnan(avgC_df.T[ip][ip_m])) and (
                             ((oneC_df.T[ip][m] == 1) or np.isnan(oneC_df.T[ip][m])) and (
                             (oneC_df.T[ip][ip_m] == 1) or np.isnan(oneC_df.T[ip][ip_m])) and (
                                     (avgC_df.T[ip][m] == 1) or np.isnan(avgC_df.T[ip][m])) and (
                                     (avgC_df.T[ip][ip_m] == 1) or np.isnan(avgC_df.T[ip][ip_m])))
        if is_ip_best:
            key = 'is_ip_best'
            add_val(patterns_dic, key, eps, algo)

        is_margin_better_ip = ((oneC_df.T[m][ip] == 1) and (avgC_df.T[m][ip] == 1)) or (
                (oneC_df.T[m][ip] == 1) and np.isnan(avgC_df.T[m][ip])) or (
                                      np.isnan(oneC_df.T[m][ip]) and (avgC_df.T[m][ip] == 1))
        if is_margin_better_ip:
            key = 'is_margin_better_ip'
            add_val(patterns_dic, key, eps, algo)
        """
        is_ip_m_best = (~np.isnan(oneC_df.T[ip_m][m]) or ~np.isnan(oneC_df.T[ip_m][ip]) or ~np.isnan(
            avgC_df.T[ip_m][m]) or ~np.isnan(avgC_df.T[ip_m][ip])) and (
                               ((oneC_df.T[ip_m][m] == 1) or np.isnan(oneC_df.T[ip_m][m])) and (
                               (oneC_df.T[ip_m][ip] == 1) or np.isnan(oneC_df.T[ip_m][ip])) and (
                                       (avgC_df.T[ip_m][m] == 1) or np.isnan(avgC_df.T[ip_m][m])) and (
                                       (avgC_df.T[ip_m][ip] == 1) or np.isnan(avgC_df.T[ip_m][ip])))
        """

        is_ip_m_best = (oneC_df.T[ip_m][m] == 1) and (oneC_df.T[ip_m][ip] == 1) and (avgC_df.T[ip_m][m] == 1) and (
                    avgC_df.T[ip_m][ip] == 1)
        if is_ip_m_best:
            key = 'is_ip_m_best'
            add_val(patterns_dic, key, eps, algo)

        is_ip_not_expected = is_main_pattern and (oneC_df.T[ip][ip_m] == 1) or (avgC_df.T[ip][ip_m] == 1)
        if is_ip_not_expected:
            key = 'is_ip_not_expected'
            add_val(patterns_dic, key, eps, algo)
        pass
    pass

In [ ]:
    
    for key in patterns_dic.keys():
        if len(patterns_dic[key]) > 0:
            str_ = '{}\n\t{}'.format(key, patterns_dic[key])
            print(str_)
            #f.write(str_ + '\n')
        pass


# Printing patterns to file

In [ ]:
"""
with open('analysis-results/patterns/{}'.format(dataset_name), 'a') as f:

    for key in patterns_dic.keys():
        if len(patterns_dic[key]) > 0:
            str_ = '{}\n\t{}'.format(key, patterns_dic[key])
            print(str_)
            f.write(str_ + '\n')
        pass
"""